<a href="https://colab.research.google.com/github/nikolasleeb/INFO523_FinalProject/blob/main/FuelEconomy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [50]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
import glob, os

# Attach Data

In [51]:
'''Connects to Google Drive to load in data'''

# '''
# Connect to and mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Find the dataset in google drive and assign path to variable called 'Match'
matches = glob.glob('/content/drive/MyDrive/**/FuelEconomy.xlsx', recursive=True)
print("Found paths:", matches)

# Using 'Match' load in the dataset using the path
path = matches[0]
fueleconomy = pd.read_excel(path, engine='openpyxl')
fueleconomy.head()
# '''

'''
# Load dataset from local file for demonstration purposes
fueleconomy = pd.read_excel('FuelEconomy.xlsx', engine='openpyxl')
fueleconomy.head()
# '''


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found paths: ['/content/drive/MyDrive/DataMiningFall25/INFO 523 Final Project/Datasets/FuelEconomy/FuelEconomy.xlsx']


"\n# Load dataset from local file for demonstration purposes\nfueleconomy = pd.read_excel('FuelEconomy.xlsx', engine='openpyxl')\nfueleconomy.head()\n# "

In [52]:
colors = { "fueleconomy": "royalblue" }

In [53]:
''' check for null and missing values'''
fueleconomy.isnull().sum()

,0
"Fuel economy, mpg",0
1980,0
1985,0
1990,0
1991,0
1992,0
1993,0
1994,0
1995,0
1996,0


# Cleaning and Transforming

In [54]:
''' Transpose the table so that years become the first column '''

id_col = fueleconomy.columns[0]            # e.g. "Fuel economy, mpg"
fueleconomy = fueleconomy.round(2)
fueleconomy = fueleconomy.set_index(id_col).T.reset_index().rename(columns={'index':'Year'})

fueleconomy.head()

"Fuel economy, mpg",Year,Car,Car SUV,Pickup truck,Van,Truck SUV
0,1980,20.01,14.58,16.52,14.14,13.19
1,1985,23.02,20.08,18.20,16.55,16.54
2,1990,23.33,18.85,17.41,17.84,16.44
3,1991,23.43,18.21,18.19,17.91,16.73
4,1992,23.07,17.84,17.48,17.92,16.20


In [55]:
''' Verify the columns, datatypes, and number of entries '''
fueleconomy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Year          34 non-null     int64  
 1   Car           34 non-null     float64
 2   Car SUV       34 non-null     float64
 3   Pickup truck  34 non-null     float64
 4   Van           34 non-null     float64
 5   Truck SUV     34 non-null     float64
dtypes: float64(5), int64(1)
memory usage: 1.7 KB


In [56]:
''' Data Cleaning for fueleconomy Data '''

# Convert Year column to Year format
fueleconomy["Year"] = pd.to_datetime(fueleconomy["Year"], format='%Y')

# Filter to keep data where the year is between 1995 and 2021 (inclusive)
fueleconomy = fueleconomy[
    (fueleconomy["Year"].dt.year >= 1995) &
    (fueleconomy["Year"].dt.year <= 2021)
]

# Preview the first 10 rows
fueleconomy.head(10)

"Fuel economy, mpg",Year,Car,Car SUV,Pickup truck,Van,Truck SUV
7,1995-01-01,23.44,17.81,16.89,18.08,16.00
8,1996-01-01,23.33,18.43,17.15,18.34,16.23
9,1997-01-01,23.37,19.23,16.84,18.18,16.13
10,1998-01-01,23.37,18.24,17.00,18.70,16.16
11,1999-01-01,23.00,18.51,16.29,18.28,16.07
12,2000-01-01,22.91,17.89,16.65,18.61,16.01
13,2001-01-01,23.05,18.83,15.95,18.04,16.41
14,2002-01-01,23.08,19.30,15.75,18.70,16.31
15,2003-01-01,23.28,19.91,16.08,18.98,16.42
16,2004-01-01,23.14,19.97,15.74,19.16,16.47


# Summary Stats

In [57]:
fueleconomy.describe()

"Fuel economy, mpg",Year,Car,Car SUV,Pickup truck,Van,Truck SUV
count,27,27.000000,27.000000,27.000000,27.000000,27.000000
mean,2008-01-01 08:53:20,26.011481,22.313333,17.261852,20.310370,19.038148
min,1995-01-01 00:00:00,22.910000,17.810000,15.740000,18.040000,16.000000
25%,2001-07-02 12:00:00,23.315000,19.265000,16.230000,18.700000,16.360000
50%,2008-01-01 00:00:00,24.270000,21.190000,16.900000,19.820000,18.190000
75%,2014-07-02 12:00:00,28.700000,24.790000,18.425000,21.465000,21.765000
max,2021-01-01 00:00:00,31.730000,30.500000,19.400000,26.210000,24.000000
std,NaN,3.176431,3.643223,1.211652,1.975641,2.878855


In [58]:
''' View skewness and kurtosis of fueleconomy data '''

# List of columns to analyze
columns_to_analyze = ['Car', 'Car SUV', 'Pickup truck', 'Van', 'Truck SUV']

# Compute skewness and kurtosis for each specified column
for col in columns_to_analyze:
    if col in fueleconomy.columns:
        col_skew = skew(fueleconomy[col], nan_policy='omit')
        col_kurt = kurtosis(fueleconomy[col], nan_policy='omit')
        # Display results
        print(f"{col} — Skewness: {col_skew:.2f}, Kurtosis: {col_kurt:.2f}")
    else:
        print(f"Warning: Column '{col}' not found in the DataFrame.")

Car — Skewness: 0.59, Kurtosis: -1.22
Car SUV — Skewness: 0.56, Kurtosis: -0.81
Pickup truck — Skewness: 0.52, Kurtosis: -1.12
Van — Skewness: 1.04, Kurtosis: 0.96
Truck SUV — Skewness: 0.41, Kurtosis: -1.37


In [59]:
''' View fueleconomy date range and total years of data '''

print("Earliest date:", fueleconomy["Year"].min())
print("Latest date:", fueleconomy["Year"].max())
print("Total months of data:", len(fueleconomy))

Earliest date: 1995-01-01 00:00:00
Latest date: 2021-01-01 00:00:00
Total months of data: 27


# Cars Summary

In [60]:
''' Calculate and print average yearly fueleconomy, standard deviation, and coefficient of variation '''

mean = fueleconomy["Car"].mean()
std = fueleconomy["Car"].std()
cv = (std / mean) * 100
print(f"Average monthly fueleconomy for Cars: {mean:,.2f}")
print(f"Standard deviation: {std:,.2f}")
print(f"Coefficient of variation: {cv:.2f}% \n")

Average monthly fueleconomy for Cars: 26.01
Standard deviation: 3.18
Coefficient of variation: 12.21% 



In [61]:
''' Create a histogram to visualize the distribution of Cars Fuel Economy '''

# Create histogram
fige1 = px.histogram(
    fueleconomy,
    x="Car",
    nbins=15,
    title="Distribution of Cars Fuel Economy",
    color_discrete_sequence=[colors["fueleconomy"]]
)

# Customize layout
fige1.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="MPG of Cars",
    yaxis_title="Frequency",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=500
)

fige1.update_traces(marker_line_color="black", marker_line_width=1, opacity=1)

fige1.show()

In [62]:
''' Outlier Detection for Car Fuel Economy using Z-Score Method and Box Plot '''

# Use an existing column name from the DataFrame (e.g., "Car")
col_name = "Car"

# Compute Z-score for the selected column
fueleconomy["Z_fueleconomy"] = (
    (fueleconomy[col_name] - fueleconomy[col_name].mean())
    / fueleconomy[col_name].std()
)

# Flag potential outliers (|Z| > 3)
fueleconomy_outliers = fueleconomy[np.abs(fueleconomy["Z_fueleconomy"]) > 3]

print(f"Detected {len(fueleconomy_outliers)} potential outliers in '{col_name}' fueleconomy.")
display(fueleconomy_outliers[["Year", col_name, "Z_fueleconomy"]])

# visual outlier detection with box plot
fige6 = px.box(
    fueleconomy,
    y=col_name,
    title=f"Outlier Detection: {col_name} Fuel Economy",
    points="all",
)
fige6.update_layout(template="plotly_white", title_x=0.5, height=500)
fige6.show()

Detected 0 potential outliers in 'Car' fueleconomy.


"Fuel economy, mpg",Year,Car,Z_fueleconomy


# Car SUV's Summary

In [63]:
''' Calculate and print average yearly fueleconomy, standard deviation, and coefficient of variation '''

mean = fueleconomy["Car SUV"].mean()
std = fueleconomy["Car SUV"].std()
cv = (std / mean) * 100
print(f"Average monthly fueleconomy for Car SUV's: {mean:,.2f}")
print(f"Standard deviation: {std:,.2f}")
print(f"Coefficient of variation: {cv:.2f}% \n")

Average monthly fueleconomy for Car SUV's: 22.31
Standard deviation: 3.64
Coefficient of variation: 16.33% 



In [64]:
''' Create a histogram to visualize the distribution of Car SUV's Fuel Economy '''

# Create histogram
fige2 = px.histogram(
    fueleconomy,
    x="Car SUV",
    nbins=14,
    title="Distribution of Car SUV's Fuel Economy",
    color_discrete_sequence=[colors["fueleconomy"]]
)

# Customize layout
fige2.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="MPG of Car SUV's",
    yaxis_title="Frequency",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=500
)

fige2.update_traces(marker_line_color="black", marker_line_width=1, opacity=1)

fige2.show()

In [65]:
''' Outlier Detection for Car Fuel Economy using Z-Score Method and Box Plot '''

# Use an existing column name from the DataFrame (e.g., "Car")
col_name = "Car SUV"

# Compute Z-score for the selected column
fueleconomy["Z_fueleconomy"] = (
    (fueleconomy[col_name] - fueleconomy[col_name].mean())
    / fueleconomy[col_name].std()
)

# Flag potential outliers (|Z| > 3)
fueleconomy_outliers = fueleconomy[np.abs(fueleconomy["Z_fueleconomy"]) > 3]

print(f"Detected {len(fueleconomy_outliers)} potential outliers in '{col_name}' fueleconomy.")
display(fueleconomy_outliers[["Year", col_name, "Z_fueleconomy"]])

# visual outlier detection with box plot
fige7 = px.box(
    fueleconomy,
    y=col_name,
    title=f"Outlier Detection: {col_name} Fuel Economy",
    points="all",
)
fige7.update_layout(template="plotly_white", title_x=0.5, height=500)
fige7.show()

Detected 0 potential outliers in 'Car SUV' fueleconomy.


"Fuel economy, mpg",Year,Car SUV,Z_fueleconomy


# Pickup truck's Summary

In [66]:
''' Calculate and print average yearly fueleconomy, standard deviation, and coefficient of variation '''

mean = fueleconomy["Pickup truck"].mean()
std = fueleconomy["Pickup truck"].std()
cv = (std / mean) * 100
print(f"Average monthly fueleconomy for Pickup trucks: {mean:,.2f}")
print(f"Standard deviation: {std:,.2f}")
print(f"Coefficient of variation: {cv:.2f}% \n")

Average monthly fueleconomy for Pickup trucks: 17.26
Standard deviation: 1.21
Coefficient of variation: 7.02% 



In [67]:
''' Create a histogram to visualize the distribution of Pickup trucks Fuel Economy '''

# Create histogram
fige3 = px.histogram(
    fueleconomy,
    x="Pickup truck",
    nbins=5,
    title="Distribution of Pickup Truck's Fuel Economy",
    color_discrete_sequence=[colors["fueleconomy"]]
)

# Customize layout
fige3.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="MPG of Pickup truck's",
    yaxis_title="Frequency",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=500
)

fige3.update_traces(marker_line_color="black", marker_line_width=1, opacity=1)

fige3.show()

In [68]:
''' Outlier Detection for Car Fuel Economy using Z-Score Method and Box Plot '''

# Use an existing column name from the DataFrame (e.g., "Car")
col_name = "Pickup truck"

# Compute Z-score for the selected column
fueleconomy["Z_fueleconomy"] = (
    (fueleconomy[col_name] - fueleconomy[col_name].mean())
    / fueleconomy[col_name].std()
)

# Flag potential outliers (|Z| > 3)
fueleconomy_outliers = fueleconomy[np.abs(fueleconomy["Z_fueleconomy"]) > 3]

print(f"Detected {len(fueleconomy_outliers)} potential outliers in '{col_name}' fueleconomy.")
display(fueleconomy_outliers[["Year", col_name, "Z_fueleconomy"]])

# visual outlier detection with box plot
fige8 = px.box(
    fueleconomy,
    y=col_name,
    title=f"Outlier Detection: {col_name} Fuel Economy",
    points="all",
)
fige8.update_layout(template="plotly_white", title_x=0.5, height=500)
fige8.show()

Detected 0 potential outliers in 'Pickup truck' fueleconomy.


"Fuel economy, mpg",Year,Pickup truck,Z_fueleconomy


# Truck SUV's Summary

In [69]:
''' Calculate and print average yearly fueleconomy, standard deviation, and coefficient of variation '''

mean = fueleconomy["Truck SUV"].mean()
std = fueleconomy["Truck SUV"].std()
cv = (std / mean) * 100
print(f"Average monthly fueleconomy for Truck SUVs': {mean:,.2f}")
print(f"Standard deviation: {std:,.2f}")
print(f"Coefficient of variation: {cv:.2f}%")

Average monthly fueleconomy for Truck SUVs': 19.04
Standard deviation: 2.88
Coefficient of variation: 15.12%


In [70]:
''' Create a histogram to visualize the distribution of Truck SUV's Fuel Economy '''
# fix number of bins or how they bins

# Create histogram
fige5 = px.histogram(
    fueleconomy,
    x="Truck SUV",
    nbins=10,
    title="Distribution of Truck SUV's Fuel Economy",
    color_discrete_sequence=[colors["fueleconomy"]]
)

# Customize layout
fige5.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="MPG of Truck SUV's",
    yaxis_title="Frequency",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=500
)

fige5.update_traces(marker_line_color="black", marker_line_width=1, opacity=1)

fige5.show()

In [71]:
''' Outlier Detection for Car Fuel Economy using Z-Score Method and Box Plot '''

# Use an existing column name from the DataFrame (e.g., "Car")
col_name = "Truck SUV"

# Compute Z-score for the selected column
fueleconomy["Z_fueleconomy"] = (
    (fueleconomy[col_name] - fueleconomy[col_name].mean())
    / fueleconomy[col_name].std()
)

# Flag potential outliers (|Z| > 3)
fueleconomy_outliers = fueleconomy[np.abs(fueleconomy["Z_fueleconomy"]) > 3]

print(f"Detected {len(fueleconomy_outliers)} potential outliers in '{col_name}' fueleconomy.")
display(fueleconomy_outliers[["Year", col_name, "Z_fueleconomy"]])

# visual outlier detection with box plot
fige9 = px.box(
    fueleconomy,
    y=col_name,
    title=f"Outlier Detection: {col_name} Fuel Economy",
    points="all",
)
fige9.update_layout(template="plotly_white", title_x=0.5, height=500)
fige9.show()

Detected 0 potential outliers in 'Truck SUV' fueleconomy.


"Fuel economy, mpg",Year,Truck SUV,Z_fueleconomy


# Van's Summary

In [72]:
''' Calculate and print average yearly fueleconomy, standard deviation, and coefficient of variation '''

mean = fueleconomy["Van"].mean()
std = fueleconomy["Van"].std()
cv = (std / mean) * 100
print(f"Average monthly fueleconomy for Vans: {mean:,.2f}")
print(f"Standard deviation: {std:,.2f}")
print(f"Coefficient of variation: {cv:.2f}% \n")

Average monthly fueleconomy for Vans: 20.31
Standard deviation: 1.98
Coefficient of variation: 9.73% 



In [73]:
''' Create a histogram to visualize the distribution of Vans Fuel Economy '''

# Create histogram
fige4 = px.histogram(
    fueleconomy,
    x="Van",
    nbins=10,
    title="Distribution of Van's Fuel Economy",
    color_discrete_sequence=[colors["fueleconomy"]]
)

# Customize layout
fige4.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="MPG of Van's",
    yaxis_title="Frequency",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=500
)

fige4.update_traces(marker_line_color="black", marker_line_width=1, opacity=1)

fige4.show()

In [74]:
''' Outlier Detection for Car Fuel Economy using Z-Score Method and Box Plot '''

# Use an existing column name from the DataFrame
col_name = "Van"

# Compute Z-score for the selected column
fueleconomy["Z_fueleconomy"] = (
    (fueleconomy[col_name] - fueleconomy[col_name].mean())
    / fueleconomy[col_name].std()
)

# Flag potential outliers (|Z| > 3)
fueleconomy_outliers = fueleconomy[np.abs(fueleconomy["Z_fueleconomy"]) > 3]

print(f"Detected {len(fueleconomy_outliers)} potential outliers in '{col_name}' fueleconomy.")
display(fueleconomy_outliers[["Year", col_name, "Z_fueleconomy"]])

# visual outlier detection with box plot
fige10 = px.box(
    fueleconomy,
    y=col_name,
    title=f"Outlier Detection: {col_name} Fuel Economy",
    points="all",
)
fige10.update_layout(template="plotly_white", title_x=0.5, height=500)
fige10.show()

Detected 0 potential outliers in 'Van' fueleconomy.


"Fuel economy, mpg",Year,Van,Z_fueleconomy


# Line Graph

In [75]:
''' create a line graph showing trends over time for each vehicle type '''

fig_line = px.line(
    fueleconomy,
    x="Year",
    y=["Car", "Car SUV", "Pickup truck", "Van", "Truck SUV"],
    title="Fuel Economy Trends Over Time by Vehicle Type",
    labels={"value": "Fuel Economy (MPG)", "variable": "Vehicle Type"},
    color_discrete_sequence=px.colors.qualitative.Set1
)
fig_line.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="Year",
    yaxis_title="Fuel Economy (MPG)",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=600
)
fig_line.show()